In [ ]:
# Parameters
MESH_DENSITY = 10
TIME_STEP_LENGTH = 0.01
N_TIME_STEPS = 100

INLET_VELOCITY = 1.0 # m/s

# Fluid properties
DENSITY = 1 # kg/m^3
VISCOSITY =  0.01 #8.9 * 10**-4 #0.01 # Pa*s
THERMAL_CONDUCTIVITY = 0.1 #0.606 # W/(m * K)
SPECIFIC_HEAT_CAPACITY =  1 #4180.0 # J/(kg * K)

# ?? Can be removed
USE_VELOCITY_LINEAR_SOLVER = True
USE_PRESSURE_LINEAR_SOLVER = True
USE_HEAT_LINEAR_SOLVER = True


In [ ]:
# Define mesh
import fenics as fe
import mshr as ms
import matplotlib.pyplot as plt

# Create the mesh
box = ms.Rectangle(fe.Point(0.0, 0.0), fe.Point(1.0, 1.0))
hole = ms.Rectangle(fe.Point(0.2,0.4),fe.Point(0.8,0.6)) # 
domain = box 

mesh = ms.generate_mesh(domain, MESH_DENSITY)

fig, ax = plt.subplots(figsize=(4,4))
ax.set_xlabel('x')
ax.set_ylabel('y')
fe.plot(mesh, title='Mesh')
plt.show()

print(mesh.num_cells())
fe.set_log_active(False)

In [ ]:
# Define the function spaces
# Taylor-Hood Elements. The order of the function space for the pressure has
# to be one order lower than for the velocity
velocity_function_space = fe.VectorFunctionSpace(mesh, "Lagrange", 3)
pressure_function_space = fe.FunctionSpace(mesh, "Lagrange", 2)

# Define the trial and test functions
u = fe.TrialFunction(velocity_function_space)
p = fe.TrialFunction(pressure_function_space)
v = fe.TestFunction(velocity_function_space)
q = fe.TestFunction(pressure_function_space)

# Heat eq
heat_function_space = fe.FunctionSpace(mesh,'Lagrange', 2)
t = fe.TrialFunction(heat_function_space)
w = fe.TestFunction(heat_function_space)

In [ ]:
# Define the Boundary Condition
def top_boundary(x, on_boundary): return on_boundary and fe.near(x[1], 1.0)
def bottom_boundary(x, on_boundary): return on_boundary and fe.near(x[1], 0.0)
def left_boundary(x, on_boundary): return on_boundary and fe.near(x[0], 0.0)
def right_boundary(x, on_boundary): return on_boundary and fe.near(x[0], 1.0)

inflow_sin = fe.Expression(('pow(sin(2*pi*x[0]),2)+0.001', 0.0), degree=2)
inflow = fe.Constant((INLET_VELOCITY, 0.0))
noslip = fe.Constant((0.0, 0.0))

velocity_boundary_conditions = [
    fe.DirichletBC(velocity_function_space, inflow, left_boundary),
    fe.DirichletBC(velocity_function_space, noslip, right_boundary),
    fe.DirichletBC(velocity_function_space, noslip, top_boundary),
    fe.DirichletBC(velocity_function_space, noslip, bottom_boundary),
]

pressure_boundary_conditions = [
    fe.DirichletBC(pressure_function_space, 0.0, left_boundary),
    fe.DirichletBC(pressure_function_space, 0.0, right_boundary),
    fe.DirichletBC(pressure_function_space, 0.0, top_boundary),
    fe.DirichletBC(pressure_function_space, 0.0, bottom_boundary)
]

heat_boundary_conditions = [
    fe.DirichletBC(heat_function_space, 0.0, left_boundary),
    fe.DirichletBC(heat_function_space, 0.0, right_boundary),
    fe.DirichletBC(heat_function_space, 0.0, top_boundary),
    fe.DirichletBC(heat_function_space, 0.0, bottom_boundary)
]


In [ ]:
# Define the solution fields involved
u_prev = fe.Function(velocity_function_space) # velocity at previous time step, used for time integration
u_tent = fe.Function(velocity_function_space) # velocity before pressure correction
u_next = fe.Function(velocity_function_space) # velocity after pressure correction, at current time step
p_next = fe.Function(pressure_function_space) # pressure at current time step
t_prev = fe.Function(heat_function_space)
t_next = fe.Function(heat_function_space)


In [ ]:
# Set initial conditions
t_prev.interpolate(
    fe.Expression(
        '60 * (0.5*(1 - tanh(8*(sqrt(pow(x[0]-0.5, 2) + pow(x[1]-0.5, 2)) - 0.1))))',
     degree=1)
)
u_prev.interpolate(
    fe.Expression(
    ('sin(x[1])', 'cos(x[0])'),
    degree=2
    )
)
fe.plot(t_prev)

In [ ]:
# Compute weak forms

dt = fe.Constant(TIME_STEP_LENGTH)
rho = fe.Constant(DENSITY)
nu = fe.Constant(VISCOSITY / DENSITY)
kappa = fe.Constant(THERMAL_CONDUCTIVITY/(DENSITY*SPECIFIC_HEAT_CAPACITY))

print('nu:',VISCOSITY / DENSITY)
print('alpha:',THERMAL_CONDUCTIVITY/(DENSITY*SPECIFIC_HEAT_CAPACITY))

# Weak form of the momentum equation
momentum_residuum = ( #? force term
    (1.0 / dt) * fe.inner(u - u_prev, v) * fe.dx
    +
    fe.inner(fe.grad(u_prev) * u_prev, v) * fe.dx
    +
    nu * fe.inner(fe.grad(u), fe.grad(v)) * fe.dx)


# Weak form of the pressure poisson problem
pressure_poisson_residuum = (
    fe.inner(fe.grad(p), fe.grad(q)) * fe.dx
    +
    (rho / dt) * fe.div(u_tent) * q * fe.dx
)

# Weak form of the velocity update equation
velocity_update_residuum = (
    fe.inner(u, v) * fe.dx
    -
    fe.inner(u_tent, v) * fe.dx
    +
    (dt/rho) * fe.inner(fe.grad(p_next), v) * fe.dx)


heat_residuum = (
    (1.0 / dt) * fe.inner(t - t_prev, w) * fe.dx
    +
    fe.inner(u_next, fe.grad(t_prev)) * w * fe.dx 
    +
    kappa * fe.inner(fe.grad(t), fe.grad(w)) * fe.dx
)


In [ ]:
# Assemble system matricies
momentum_lhs = fe.lhs(momentum_residuum)
momentum_rhs = fe.rhs(momentum_residuum)
momentum_A = fe.assemble(momentum_lhs)

pressure_poisson_lhs = fe.lhs(pressure_poisson_residuum)
pressure_poisson_rhs = fe.rhs(pressure_poisson_residuum)
pressure_poisson_A = fe.assemble(pressure_poisson_lhs)

velocity_update_lhs = fe.lhs(velocity_update_residuum)
velocity_update_rhs = fe.rhs(velocity_update_residuum)
velocity_update_A = fe.assemble(velocity_update_lhs)

heat_lhs = fe.lhs(heat_residuum)
heat_rhs = fe.rhs(heat_residuum)
heat_A = fe.assemble(heat_lhs)

In [ ]:
# Time loop
from tqdm import tqdm  # for progress bar

for t in tqdm(range(N_TIME_STEPS)):
    # (1) Solve for tentative velocity
    if USE_VELOCITY_LINEAR_SOLVER:
        momentum_b = fe.assemble(momentum_rhs)
        [bc.apply(momentum_A, momentum_b)
         for bc in velocity_boundary_conditions]
        fe.solve(
            momentum_A,
            u_tent.vector(),
            momentum_b,
            "gmres",
            "ilu",
        )
    else:
        fe.solve(
            momentum_lhs == momentum_rhs,
            u_tent,
            velocity_boundary_conditions
        )

    # (2) Solve for the pressure
    if USE_PRESSURE_LINEAR_SOLVER:
        pressure_poisson_b = fe.assemble(pressure_poisson_rhs)
        [bc.apply(pressure_poisson_A, pressure_poisson_b)
         for bc in pressure_boundary_conditions]
        fe.solve(
            pressure_poisson_A,
            p_next.vector(),
            pressure_poisson_b,
            "gmres",
            "amg",
        )
    else:
        fe.solve(
            pressure_poisson_lhs == pressure_poisson_rhs,
            p_next,
            pressure_boundary_conditions
        )

    # (3) Correct the velocities to be incompressible
    if USE_VELOCITY_LINEAR_SOLVER:
        velocity_update_b = fe.assemble(velocity_update_rhs)
        [bc.apply(velocity_update_A, velocity_update_b)
         for bc in velocity_boundary_conditions]
        fe.solve(
            velocity_update_A,
            u_next.vector(),
            velocity_update_b,
            "gmres",
            "ilu",
        )
    else:
        fe.solve(
            velocity_update_lhs == velocity_update_rhs,
            u_next,
            velocity_boundary_conditions)

    # (4) Solve heat advection-diffusion equation
    if USE_HEAT_LINEAR_SOLVER:
        heat_b = fe.assemble(heat_rhs)
        [bc.apply(heat_A, heat_b) for bc in heat_boundary_conditions]
        fe.solve(
            heat_A,
            t_next.vector(),
            heat_b,
            "gmres",
            "amg",
        )
    else:
        fe.solve(
            heat_lhs == heat_rhs,
            t_next,
            heat_boundary_conditions,
        )

    # Advance in time
    t_prev.assign(t_next)
    u_prev.assign(u_next)


In [ ]:
# Plot results
import matplotlib.pyplot as plt

# print(p_next.compute_vertex_values(mesh))
# print(u_next.compute_vertex_values(mesh))
plt.figure(figsize=(7, 7),dpi=100)

pressure_plot = fe.plot(p_next, 
                        cmap='jet',
                        levels=50)
plt.colorbar(pressure_plot, label='Pressure')
velocity_plot = fe.plot(u_next, 
                        edgecolor='black',
                        linewidth=0.5, 
                        minshaft=1, 
                        minlength=1.5,
                        title='Velocity/Pressure')
fe.plot(mesh, linewidth=0.1)
plt.show()

In [ ]:
# Plot heat
import cmocean

plt.figure(figsize=(7, 7), dpi=100)
heat_plot = fe.plot(
    t_next, 
    cmap=cmocean.cm.thermal,
    levels=50)
fe.plot(t_next,mode='contour',levels=10,cmap='coolwarm')
plt.colorbar(heat_plot, label='Heat, $\degree C$',)
fe.plot(mesh, linewidth=0.1)
fe.plot(u_next,edgecolor='white',cmap=cmocean.cm.speed)
plt.show()

In [ ]:
# Plot curl
import cmocean

plt.figure(figsize=(7, 7), dpi=100)
curl_plot = fe.plot(fe.curl(u_next), 
                    cmap=cmocean.cm.balance,
                    levels=50,
                    title='$\operatorname{curl} \mathbf{u}$')
plt.colorbar(curl_plot, label='curl',)
fe.plot(mesh, linewidth=0.1)
fe.plot(u_next,cmap=cmocean.cm.speed)
plt.show()